# Enrichment Analysis on LCI Predictions

Analyzes the enrichment of proteins predicted by LCI

In [47]:
%load_ext autoreload
%autoreload 2

import sys, os
sys.path.append(os.path.abspath(os.path.join('..')))

import numpy as np
import matplotlib.pyplot as plt 
import torch
import numpy as np
import pandas as pd
import networkx as nx
from tqdm import tqdm_notebook as tqdm
import goatools
from goatools.base import download_go_basic_obo, download_ncbi_associations
from goatools.obo_parser import GODag
from goatools.associations import read_ncbi_gene2go
from goatools.go_enrichment import GOEnrichmentStudy


from dpp.methods.lci.lci_method import LCIModule
from dpp.data.network import PPINetwork
from dpp.util import Params
from dpp.data.associations import load_diseases

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Loading Data
Load disease associations and protein-protein interaction network.

In [2]:
# load diseases
diseases_dict = load_diseases("../data/associations/disgenet-associations.csv", exclude_splits=['none'])

In [4]:
# load network
network = PPINetwork("../data/networks/bio-pathways-network.txt")
n = len(network)

## Load Predictions
Load predictions from a disease protein prediction method.

In [13]:
predictions= pd.read_csv("../experiments/bio-pathways/dpp_predict/lci/predictions.csv", index_col=0)

In [ ]:
# The directory of the LCI experiment to use
experiment_dir = "../../experiments/disgenet/bio-pathways/dpp-10/learned_cn/cv/nodup7-cc7"

In [ ]:
params = Params(os.path.join(experiment_dir, "params.json"))

In [ ]:
model = LCIModule(params, ppi_network_adj)

In [ ]:
# diseases used in enrichment analysis
disease_ids = ["C1862314"] #"C0006845", "C0086795", "C0028326", "C0855740", "C0008325"]
diseases = map(diseases_dict.get, disease_ids)

In [ ]:
# GPU settings
cuda = True

In [ ]:
# apply 
disease_to_ranks = {}
for disease in tqdm(diseases):
    # load correct model from cross validation
    weights_path = os.path.join(experiment_dir, "models", "model_{}.tar".format(disease.split))
    model.load_state_dict(torch.load(weights_path))
    model.eval()
    if cuda:
        model.cuda()
    
    disease_nodes = disease.to_node_array(protein_to_node)
    
    # prepare input matrix
    N, _ = ppi_network_adj.shape
    X = torch.zeros(1, N)
    X[0, disease_nodes] = 1
    if cuda:
        X = X.cuda()
    
    Y = model(X)
    scores = Y.cpu().detach().numpy().squeeze()
    scores[disease_nodes] = 0
    ranked_nodes = np.argsort(scores)
    disease_to_ranks[disease.id] = ranked_nodes
    

## Load Enrichment Analysis
Prepare an enrichment study

In [14]:
# download GOA obo file
obo_fname = download_go_basic_obo();

  wget.download(http://purl.obolibrary.org/obo/go/go-basic.obo out=go-basic.obo)


In [16]:
# load gene ontology
obodag = GODag("../data/go/go-basic.obo")

../data/go/go-basic.obo: fmt(1.2) rel(2019-01-19) 47,379 GO Terms


In [17]:
geneid2go = read_ncbi_gene2go("../data/go/gene2go.txt", taxids=[9606])

  20,385 items READ: ../data/go/gene2go.txt


In [23]:
goeaobj = GOEnrichmentStudy(network.get_proteins(), # List of mouse protein-coding genes
                            geneid2go, # geneid/GO associations
                            obodag, # Ontologies
                            propagate_counts = True,
                            alpha = 0.05, # default significance cut-off
                            methods = ['fdr_bh']) # defult multipletest correction method

fisher module not installed.  Falling back on scipy.stats.fisher_exact


Propagating term counts to parents ..


 76% 16,420 of 21,557 population items found in association


## Perform Enrichment Analysis
Perform an enrichment analysis on one disease. 

In [69]:
predictions.loc[disease_id].sort_values(ascending=False).index

Index(['26532', '91703', '10121', '80228', '5967', '389203', '55709', '832564',
       '60684', '134391',
       ...
       '84450', '126075', '389799', '27111', '220042', '84236', '10998',
       '83475', '8435', '84293'],
      dtype='object', length=21557)

In [65]:
disease_id = "C1862314"
disease_proteins = diseases_dict[disease_id].proteins
disease_proteins

{1643,
 2068,
 2071,
 2072,
 2073,
 2956,
 3265,
 3845,
 4000,
 4292,
 4436,
 4893,
 5395,
 5429,
 5727,
 5921,
 6608,
 7508,
 8643,
 9401,
 11322,
 23405,
 51684}

In [66]:
disease_id = "C1862314"
disease_proteins = diseases_dict[disease_id].proteins
pred_proteins = set(map(int, predictions.loc[disease_id].sort_values(ascending=False).index[:len(disease_proteins)]))
disease_results = goeaobj.run_study(disease_proteins)

pred_results = goeaobj.run_study(pred_proteins)

100%     23 of     23 study items found in association
100%     23 of     23 study items found in population(21557)
Calculating 21,968 uncorrected p-values using fisher_scipy_stats
  21,968 GO terms are associated with 16,420 of 21,557 population items
   1,564 GO terms are associated with     23 of     23 study items
     334 GO terms found significant (< 0.05=alpha) after multitest correction: statsmodels fdr_bh
 78%     18 of     23 study items found in association
100%     23 of     23 study items found in population(21557)
Calculating 21,968 uncorrected p-values using fisher_scipy_stats
  21,968 GO terms are associated with 16,420 of 21,557 population items
   1,067 GO terms are associated with     18 of     23 study items
       0 GO terms found significant (< 0.05=alpha) after multitest correction: statsmodels fdr_bh


In [85]:
k = 10
disease_top_k = sorted(disease_results, key=lambda x: x.p_fdr_bh)[:k]
disease_significant = [r for r in disease_results if r.p_fdr_bh < 0.02]

pred_top_k = sorted(pred_results, key=lambda x: x.p_fdr_bh)[:k]
pred_significant = [r for r in pred_results if r.p_fdr_bh < 0.02]

In [86]:
intersection = set([result.goterm.name for result in disease_top_k]) & set([result.goterm.name for result in pred_top_k])
union = set([result.goterm.name for result in disease_top_k]) | set([result.goterm.name for result in pred_top_k])

print("Jaccard Similarity: {}".format(1.0*len(intersection)/len(union)))
print(intersection)

Jaccard Similarity: 0.25
set(['cellular response to DNA damage stimulus', 'DNA repair', 'damaged DNA binding', 'DNA metabolic process'])


In [87]:
intersection = set([result.goterm.name for result in pred_significant]) & set([result.goterm.name for result in disease_significant])
union = set([result.goterm.name for result in pred_significant]) | set([result.goterm.name for result in disease_significant])

print("Jaccard Similarity: {}".format(1.0*len(intersection)/len(union)))
print(intersection)

Jaccard Similarity: 0.290243902439
set(['cell cycle process', 'oxidized DNA binding', 'negative regulation of DNA metabolic process', 'protein-DNA complex subunit organization', 'organic substance metabolic process', 'mitotic cell cycle checkpoint', 'regulation of DNA replication', 'nucleobase-containing compound metabolic process', 'regulation of nucleobase-containing compound metabolic process', 'intracellular organelle part', 'DNA repair complex', 'regulation of smoothened signaling pathway', 'UV protection', 'cellular response to abiotic stimulus', 'DNA repair', 'intrinsic apoptotic signaling pathway in response to DNA damage', 'regulation of helicase activity', 'nucleoside-triphosphatase activity', 'cellular aromatic compound metabolic process', 'organic cyclic compound metabolic process', 'nucleic acid metabolic process', 'pyrophosphatase activity', 'apoptotic signaling pathway', 'ATPase activity, coupled', 'mismatch repair', 'organic cyclic compound binding', 'hydrolase activity

In [ ]:
# compute jaccard similarity for all diseases
# find ten diseases 